# Pipeline scaling

In this notebook, we are going to scale the different steps of our Machine Learning process by making a pipeline.

In [1]:
# === System imports ===


# === Third-party import ===
from  sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler

# === Local imports ===
import preprocessors as pp
import config

## Making the Piepeline

The Pipeline steps are:

1. Numerical: Remove outliers IQR
2. Numerical: Log transform variables
3. Categorical: Add 'Rare' label
4. Categorical: One-hot-encoder
5. Categorical: Add additional columns
6. Feature Scaling
7. Lasso model


In [ ]:
pipe = Pipeline(
    [
        ('outliers_remover',
            pp.OutliersRemoval(variables=config.NUMERICAL_VARIABLES)),
        ('log_transformer',
            pp.LogTransformer(variables=config.VARIABLES_TO_LOG_TRANSFORM)),
        ('rare_label_encoder',
            pp.RareLabelCategoricalEncode(variables=config.CATEGORICAL_VARIABLES)),
        ('one_hot_encoder',
            pp.OneHotEncoder(variables=variables=config.CATEGORICAL_VARIABLES)),
        ('scaler', MinMaxScaler()),
        ('model', Lasso(alpha=0.005, random_state=0))
    ]
)